In [13]:
from ultralytics import YOLO
import cv2
# Загрузка предобученной модели YOLOv8
model = YOLO('yolov8n.pt')  # 'n' — nano версия (самая легкая)

In [14]:
image = cv2.imread('ships_3.jpg')

results = model(image)

# Инициализация счетчика кораблей
tv_count = 0

# Перебираем все обнаруженные объекты
for result in results:
    for box in result.boxes:
        class_id = int(box.cls)
        if class_id == 8:  # Нужный класс в COCO
            tv_count += 1

# Визуализация результатов с bounding boxes
annotated_image = results[0].plot()

# Добавляем текст с количеством кораблей
text = f'SHIPS Count: {tv_count}'
cv2.putText(annotated_image, text, (850, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (180, 20, 220), 2)

#cv2.imwrite('./data/tv_data/detected_tvs/detected_tvs.jpg', annotated_image)

# Показываем результат 
cv2.imshow('TV Detection', annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Выводим количество
#print(f"Найдено кораблей: {tv_count}")


0: 448x640 3 boats, 127.0ms
Speed: 6.9ms preprocess, 127.0ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)


In [21]:
%tb
from flask import Flask, request, jsonify, send_from_directory
from ultralytics import YOLO
import cv2
import tempfile
import os
from flask_cors import CORS

import shutil

# Путь к папке временных файлов
TEMP_DIR = "temporary_folder"

# Очистка папки перед запуском (опционально)
shutil.rmtree(TEMP_DIR, ignore_errors=True)
os.makedirs(TEMP_DIR, exist_ok=True)

app = Flask(__name__)
CORS(app)  # Разрешить запросы с любых доменов
model = YOLO('yolov8n.pt')

@app.route('/')
def home():
    return "Succsess"

@app.route('/process', methods=['POST'])
def process_video():
    try:
        if 'video' not in request.files:
            return jsonify({'error': 'No video uploaded'}), 400

        video_file = request.files['video']
        if video_file.filename == '':
            return jsonify({'error': 'Empty filename'}), 400

        # Сохраняем входное видео
        input_path = os.path.join(TEMP_DIR, "input_video.mp4")
        video_file.save(input_path)

        # Инициализация видео-потоков
        cap = cv2.VideoCapture(input_path)
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Выходной файл
        output_filename = "output_video.mp4"
        output_path = os.path.join(TEMP_DIR, output_filename)

        fourcc = cv2.VideoWriter_fourcc(*'avc1')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        counts = []

        # Обработка кадров
        try:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                results = model(frame, classes=[8])
                count = len(results[0].boxes)
                counts.append(count)

                annotated_frame = results[0].plot()
                out.write(annotated_frame)
        finally:
            # Гарантированное освобождение ресурсов
            cap.release()
            out.release()

        # Удаление временных файлов (опционально)
        #os.remove(input_path)
        #os.remove(output_path)

        return jsonify({
            'video_url': f'/download/{output_filename}',
            'counts': counts
        })

    except Exception as e:
        return jsonify({'error': str(e)}), 500


@app.route('/download/<filename>')
def download_file(filename):
    return send_from_directory(
        TEMP_DIR,
        filename,
        mimetype='video/mp4',  # Явно указываем MIME-тип
        as_attachment=False
    )

if __name__ == '__main__':
    app.run()

NameError: name 'annotated_frame' is not defined

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/May/2025 00:18:12] "GET / HTTP/1.1" 200 -


In [4]:
!pip install flask_cors

  Obtaining dependency information for flask_cors from https://files.pythonhosted.org/packages/85/61/4aea5fb55be1b6f95e604627dc6c50c47d693e39cab2ac086ee0155a0abd/flask_cors-5.0.1-py3-none-any.whl.metadata
